# Testing SWoTTeD on MIMIC-IV data


In [ ]:
from model.temporalPhenotyping import TemporalPhenotyping
from model.loss_metrics import *
from model.slidingWindow_model import SlidingWindow
from model.utils import success_rate
import matplotlib.pyplot as plt
import numpy as np
import torch
import pickle
import seaborn as sns

### Params

In [ ]:
R = 4           #: number of phenotype
Tw = 3          #: length of time's window

### Loading MIMIC-IV data

In [ ]:
# Indicate the path to the file containing mimic-iv data, generated using CohortConstruction.ipynb
W_, Ph_, X, params =  pickle.load(open("mimic_data.pkl","rb"))

# selecting patients with T > Tw
data = []
for i in range(len(X)):
    if X[i].shape[1] > Tw:
        data.append(X[i])
	
X = data

### Temporal phenotyping

In [ ]:
# launch the model learning
loss = eval("Bernoulli")
model = eval("SlidingWindow")
tempPheno = TemporalPhenotyping(metric=loss(), model=model())
tempPheno.fit(X, 
		rank = R, 
		temp_window_length = Tw, 
		batch_size = 50,
		n_epochs = 1000, 
		sparsity= True, 
		non_negativity = True, 
		normalization = True, 
		pheno_succession = True, 
		temp_reg = False,
		trace=True)

### Results

- Phenotypes

In [ ]:
for i in range(R):
    sns.heatmap(tempPheno.Ph[i].detach().numpy(), vmin=0, vmax=1, cmap="binary")
    plt.ylabel("Drugs")
    plt.xlabel("time")
    plt.title("result")
    plt.show()

- Reconctruction

In [ ]:
tempPheno.reconstruction()
Y = tempPheno.recons

In [ ]:
for i in range(10):
        plt.subplot(221)
        sns.heatmap(X[i], vmin=0, vmax=1, cmap="binary")
        plt.ylabel("Drugs")
        plt.xlabel("time")
        plt.title("phenotype")
        plt.subplot(222)
        sns.heatmap(Y[i].detach().numpy(), vmin=0, vmax=1, cmap="binary")
        plt.ylabel("Drugs")
        plt.xlabel("time")
        plt.title("result")
        plt.show()

### FIT Score

In [ ]:
error_X= success_rate(X, Y)
print(error_X)